In [3]:
%%capture
!pip install smolagents
!pip install arize-phoenix opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents

In [ ]:
# Run the collector in the background with the below command

!python -m phoenix.server.main serve

In [8]:
import os 

hf_token = os.getenv("HF_TOKEN")

### Setting up telemetry with Arize AI Phoenix

In [5]:
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor

endpoint = "http://0.0.0.0:6006/v1/traces"
trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

In [11]:
from smolagents import (
    CodeAgent, 
    ToolCallingAgent, 
    DuckDuckGoSearchTool, 
    VisitWebpageTool, 
    HfApiModel,
)

model = HfApiModel(token=hf_token)

search_agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool()],
    model=model, 
    name="search_agent", 
    description="This is an agent that can do web search."
)

manager_agent = CodeAgent(
    tools=[],
    model=model, 
    managed_agents=[search_agent],
)

manager_agent.run(
    "If the US keeps its 2024 growth rate, how many years will it take for the GDP to double?"
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ If the US keeps its 2024 growth rate, how many years will it take for the GDP to double?                        │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  growth_rate = search_agent(task="Find the estimated GDP growth rate for the US in 2024.")                        
  print(f"The estimated GDP growth rate for the US in 2024 is: {growth_rate}")                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭──────────────────────────────────────────── New run - search_agent ─────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'search_agent'.                                                                    │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Find the estimated GDP growth rate for the US in 2024.                                                          │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  growth_rate = 2.8  # in percentage                                                                               
  years_to_double = 70 / growth_rate                                                                               
  final_answer(years_to_double)                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 25.0

[Step 1: Duration 7.96 seconds| Input tokens: 6,970 | Output tokens: 573]

25.0

### Setting up telemetry with Langfuse

In [15]:
%%capture
!pip install smolagents
!pip install opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents

In [16]:
import base64

LANGFUSE_PUBLIC_KEY=os.getenv("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY=os.getenv("LANGFUSE_SECRET_KEY")

LANGFUSE_AUTH=base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel" # EU data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

In [17]:
from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

Attempting to instrument while already instrumented


In [19]:
from smolagents import (
    CodeAgent,
    ToolCallingAgent,
    DuckDuckGoSearchTool,
    VisitWebpageTool,
    HfApiModel,
)

model = HfApiModel()

search_agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool()],
    model=model,
    name="search_agent",
    description="This is an agent that can do web search.",
)

manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[search_agent],
)
manager_agent.run(
    "How can Langfuse be used to monitor and improve the reasoning and decision-making of smolagents when they execute multi-step tasks, like dynamically adjusting a recipe based on user feedback or available ingredients?"
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How can Langfuse be used to monitor and improve the reasoning and decision-making of smolagents when they       │
│ execute multi-step tasks, like dynamically adjusting a recipe based on user feedback or available ingredients?  │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  langfuse_info = search_agent(task="Provide a detailed explanation of how Langfuse can monitor and improve the    
  reasoning and decision-making of agents in multi-step tasks such as dynamically adjusting a recipe based on      
  user feedback or available ingredients.")                                                                        
  print(langfuse_info)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭──────────────────────────────────────────── New run - search_agent ─────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'search_agent'.                                                                    │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Provide a detailed explanation of how Langfuse can monitor and improve the reasoning and decision-making of     │
│ agents in multi-step tasks such as dynamically adjusting a recipe based on user feedback or available           │
│ ingredients.                                                                                                    │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(refined_explanation)                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 
**Using Langfuse to Monitor and Improve Smolagents for Dynamic Recipe Adjustment**

Langfuse enhances the monitoring and decision-making capabilities of smolagents in dynamic recipe adjustments 
through:

1. **Traceability and Observability**: Comprehensive observability allows tracing the decision-making process to 
identify discrepancies and performance bottlenecks.
2. **User Feedback Integration**: Collects and analyzes user feedback for continuous improvement.
3. **Human Annotation**: Facilitates expert reviews and refinements of decision-making algorithms.
4. **Continuous Optimization**: Uses performance metrics and feedback loops to iteratively enhance accuracy and 
efficiency.
5. **Advanced Tracing Features**: Includes prompt management and metadata collection for detailed process tracking 
and optimization.

This enables smolagents to dynamically adjust recipes accurately based on user feedback and available ingredients, 
improving overall user satisfaction.

[Step 3: Duration 6.76 seconds| Input tokens: 17,891 | Output tokens: 1,362]

'\n**Using Langfuse to Monitor and Improve Smolagents for Dynamic Recipe Adjustment**\n\nLangfuse enhances the monitoring and decision-making capabilities of smolagents in dynamic recipe adjustments through:\n\n1. **Traceability and Observability**: Comprehensive observability allows tracing the decision-making process to identify discrepancies and performance bottlenecks.\n2. **User Feedback Integration**: Collects and analyzes user feedback for continuous improvement.\n3. **Human Annotation**: Facilitates expert reviews and refinements of decision-making algorithms.\n4. **Continuous Optimization**: Uses performance metrics and feedback loops to iteratively enhance accuracy and efficiency.\n5. **Advanced Tracing Features**: Includes prompt management and metadata collection for detailed process tracking and optimization.\n\nThis enables smolagents to dynamically adjust recipes accurately based on user feedback and available ingredients, improving overall user satisfaction.\n'

### Building Agents That Use Code


In [20]:
from huggingface_hub import login

login()

In [24]:
from smolagents import HfApiModel, CodeAgent, DuckDuckGoSearchTool, tool

agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=HfApiModel())
agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for the best music recommendations for a party at the Wayne's mansion.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  playlist_results = web_search(query="best music playlists for a high-end party")                                 
  print(playlist_results)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Best Songs To Party: DJ's Ultimate Party Songs Playlist - 
Top40Weekly.com](https://top40weekly.com/best-songs-to-party/)
Best Songs To Party: DJ's Ultimate Party Songs Playlist If you're looking for the ultimate playlist to get your 
party started, look no further than this best party songs playlist. Whether you're hosting a birthday bash, a 
backyard barbecue, or just a night in with friends, the right music can make all the difference.

[The 75 Best Party Songs That Will Get Everyone Dancing - 
Gear4music](https://www.gear4music.com/blog/best-party-songs/)
The best party songs 1. "September" - Earth, Wind & Fire (1978) Quite possibly the best party song. An infectious 
mix of funk and soul, "September" is celebrated for its upbeat melody and "ba-dee-ya" chorus, making it a timeless 
dance favorite.

[100 Best Party Songs: The Ultimate Party Playlist](https://audiocaptain.com/best-party-songs/)
Best Party Songs: The Ultimate Party Playlist. Here's the 100 best party songs of all time you can use to create 
your ultimate party playlist in 2025. ... Invest in turntables with high-end jog wheels since these do most of the 
work. You'll also need a quality mixer to crossfade from mixing to scratching. Reading A Crowd. Aside from having 
...

[45 Songs That Get Your Event Guests on the Dance Floor Every 
Time](https://hub.theeventplannerexpo.com/entertainment/35-songs-that-get-your-event-guests-on-the-dance-floor-ever
y-time)
You'll know your client's event best, including music genre preferences and styles. But these songs are wildly 
popular among many generations and are always great to have on standby should your dance guests need a boost. Party
Songs "Flowers" by Miley Cyrus (2023) "TQG" by KAROL G & Shakira (2023) "TRUSTFALL" by P!nk (2023)

[50 Songs on Every Event Planner's Playlist - 
Eventbrite](https://www.eventbrite.com/blog/event-planning-playlist-ds00/)
Playing non-stop party tunes might be distracting for attendees or potential new clients. Instead, curate a list of
songs for corporate events with a mix of genres that keeps guests energized without disturbing the flow. Below is a
sample of a music playlist for corporate events: "24K Magic" - Bruno Mars

[Best Party Playlist: 25 High-Energy Songs for the Ultimate 
Celebration](https://audioassemble.com/best-party-playlist/)
Best Party Playlist: 25 High-Energy Songs for the Ultimate Celebration. by John Chiong July 7, 2023 • 1 min read 
Every unforgettable party starts with a killer soundtrack. Imagine walking into a room pulsing with electric 
energy, where every beat of the music resonates with your heartbeat, coaxing you toward the dance floor. This is 
the power ...

[50+ Best Songs For Your Next Party in 2025 - Aleka's 
Get-Together](https://alekasgettogether.com/top-songs-for-any-party/)
Whether you're planning a How To Host The Best Party At Home or just want to liven up your playlist, we've got you 
covered with 100 crowd-pleasing tunes. From timeless classics to modern hits, these songs are guaranteed to keep 
the energy high and the good times rolling all night long. 50 Popular Party Songs. Uptown Funk - Mark Ronson ft ...

[The Best Party Songs of All Time (Our Playlists)](https://www.ispytunes.com/post/best-party-songs)
Our playlists feature the top party songs, from timeless classics to the latest hits. Discover the best party songs
to make your event unforgettable! We have the top party songs, from timeless classics to the latest hits. ... 
Hip-hop and rap are staples for any party playlist. They offer high energy and catchy beats that keep the crowd 
lively.

[Dance-party playlist: 100 greatest songs for an epic party - Time 
Out](https://www.timeout.com/music/dance-party-playlist-100-greatest-songs-for-an-epic-party)
Dance-party playlist: 100 greatest songs for an epic party House party, BBQ, wedding party or just a bedroom disco,
we have the party playlist you need to get the place moving. Tuesday 16 July 2013

[Be

[Step 0: Duration 1.03 seconds| Input tokens: 2,085 | Output tokens: 107]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(recommended_songs)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: ['September - Earth, Wind & Fire', 'Uptown Funk - Mark Ronson ft. Bruno Mars', 'TQG - KAROL G &
Shakira', 'TRUSTFALL - P!nk', 'Flowers - Miley Cyrus', "Don't Stop Believin' - Journey", 'One More Time - Daft 
Punk', '24K Magic - Bruno Mars', 'Shape of You - Ed Sheeran', "Can't Stop the Feeling! - Justin Timberlake", 
'Billie Jean - Michael Jackson', 'Dancing Queen - ABBA', 'Rhinestone Cowboy - Neil Diamond', 'Shake It Off - Taylor
Swift', "Let's Get Ready to Rumble - Billy Joel", 'Dancing in the Dark - Bruce Springsteen', 'Best is Yet to Come -
Imagine Dragons', 'Celebration - Kool & The Gang', 'Brown Eyed Girl - Van Morrison', 'Rescue Me - The Petrified 
Woodchip Boys']

[Step 3: Duration 5.79 seconds| Input tokens: 16,894 | Output tokens: 874]

['September - Earth, Wind & Fire',
 'Uptown Funk - Mark Ronson ft. Bruno Mars',
 'TQG - KAROL G & Shakira',
 'TRUSTFALL - P!nk',
 'Flowers - Miley Cyrus',
 "Don't Stop Believin' - Journey",
 'One More Time - Daft Punk',
 '24K Magic - Bruno Mars',
 'Shape of You - Ed Sheeran',
 "Can't Stop the Feeling! - Justin Timberlake",
 'Billie Jean - Michael Jackson',
 'Dancing Queen - ABBA',
 'Rhinestone Cowboy - Neil Diamond',
 'Shake It Off - Taylor Swift',
 "Let's Get Ready to Rumble - Billy Joel",
 'Dancing in the Dark - Bruce Springsteen',
 'Best is Yet to Come - Imagine Dragons',
 'Celebration - Kool & The Gang',
 'Brown Eyed Girl - Van Morrison',
 'Rescue Me - The Petrified Woodchip Boys']

#### Using a Custom Tool to Prepare the Menu

In [28]:
@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggest a menu based on the occasion
    Args:
        occasion: The type of the occasion for the party
    """

    if occasion == "casual":
        return "Pizza, snack, and drinks"
    elif occasion == "formal":
        return "3-course dinner with wine and dessert"
    elif occasion == "superhero":
        return "Buffet with high energy and healthy food"
    else:
        return "Custom menu for butler."

agent = CodeAgent(tools=[suggest_menu], model=HfApiModel())
agent.run("Prepare a formal menu.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Prepare a formal menu.                                                                                          │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  formal_menu = suggest_menu(occasion="formal")                                                                    
  print(formal_menu)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
3-course dinner with wine and dessert

Out: None

[Step 0: Duration 3.20 seconds| Input tokens: 2,062 | Output tokens: 64]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(formal_menu)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'Starter': 'Smoked Salmon and Cucumber Canapés with Lemon Butter and Dill alternative with 
Smoked Salmon Cremieux with Crème Fraîche', 'Main Course': 'Pan-Seared Salmon with Truffle Butter Sauce served with
Roasted Asparagus and Garlic Mash', 'Dessert': 'Chocolate Lava Cake with Vanilla Ice Cream'}

[Step 3: Duration 2.34 seconds| Input tokens: 9,456 | Output tokens: 325]

{'Starter': 'Smoked Salmon and Cucumber Canapés with Lemon Butter and Dill alternative with Smoked Salmon Cremieux with Crème Fraîche',
 'Main Course': 'Pan-Seared Salmon with Truffle Butter Sauce served with Roasted Asparagus and Garlic Mash',
 'Dessert': 'Chocolate Lava Cake with Vanilla Ice Cream'}

#### Using python inside the agent

In [29]:
from smolagents import HfApiModel, CodeAgent
import numpy as np
import time
import datetime

agent = CodeAgent(model=HfApiModel(), tools=[], additional_authorized_imports=['datetime'])

agent.run(
    """
    Alfred needs to prepare for the party. Here are the tasks:
    1. Prepare the drinks - 30 mins
    2. Decorate the mansion - 60 mins 
    3. Set up the menu - 45 mins
    4. Prepare the music and playlist - 45 mins

    If we start right now, at what time will the party be ready?
    """
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Alfred needs to prepare for the party. Here are the tasks:                                                      │
│     1. Prepare the drinks - 30 mins                                                                             │
│     2. Decorate the mansion - 60 mins                                                                           │
│     3. Set up the menu - 45 mins                                                                                │
│     4. Prepare the music and playlist - 45 mins                                                                 │
│                                                                                                                 │
│     If we start right now, at what time will the party be ready?                                                │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  from datetime import datetime, timedelta                                                                         
                                                                                                                   
  # Define the time required for each task in minutes                                                              
  tasks = {                                                                                                        
      "Prepare the drinks": 30,                                                                                    
      "Decorate the mansion": 60,                                                                                  
      "Set up the menu": 45,                                                                                       
      "Prepare the music and playlist": 45                                                                         
  }                                                                                                                
                                                                                                                   
  # Calculate the total time required for all tasks                                                                
  total_time_required = sum(tasks.values())                                                                        
                                                                                                                   
  # Get the current time                                                                                           
  current_time = datetime.now()                                                                                    
                                                                                                                   
  # Calculate the time when the party will be ready                                                                
  party_ready_time = current_time + timedelta(minutes=total_time_required)                                         
                                                                                                                   
  # Print the time when the party will be ready                                                                    
  print(party_ready_time.strftime("%Y-%m-%d %H:%M:%S"))                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
2025-03-02 15:05:33

Out: None

[Step 0: Duration 12.25 seconds| Input tokens: 2,096 | Output tokens: 223]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  from datetime import datetime, timedelta                                                                         
                                                                                                                   
  # Define the time required for each task in minutes                                                              
  tasks = {                                                                                                        
      "Prepare the drinks": 30,                                                                                    
      "Decorate the mansion": 60,                                                                                  
      "Set up the menu": 45,                                                                                       
      "Prepare the music and playlist": 45                                                                         
  }                                                                                                                
                                                                                                                   
  # Calculate the total time required for all tasks                                                                
  total_time_required = sum(tasks.values())                                                                        
                                                                                                                   
  # Get the current time                                                                                           
  current_time = datetime.now()                                                                                    
                                                                                                                   
  # Calculate the time when the party will be ready                                                                
  party_ready_time = current_time + timedelta(minutes=total_time_required)                                         
                                                                                                                   
  # Provide the final answer                                                                                       
  final_answer(party_ready_time.strftime("%Y-%m-%d %H:%M:%S"))                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 2025-03-02 15:06:25

[Step 4: Duration 15.36 seconds| Input tokens: 15,112 | Output tokens: 1,040]

'2025-03-02 15:06:25'

#### Push the agent to Hub and download an agent from the Hub

In [ ]:
agent.push_to_hub('username/reponame')

alfred_agent = agent.from_hub('sergiopaniego/AlfredAgent')

alfred_agent.run("Give me the best playlist for a party at Wayne's mansion. The party idea is a 'villain masquerade' theme")  

#### Writing actions as code snippets or JSON blobs

In [30]:
from smolagents import ToolCallingAgent, HfApiModel, DuckDuckGoSearchTool

agent = ToolCallingAgent(tools=[DuckDuckGoSearchTool()], model=HfApiModel())
agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for the best music recommendations for a party at the Wayne's mansion.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'web_search' with arguments: {'query': "best music recommendations for a party at Wayne's         │
│ mansion"}                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: ## Search Results

|50 Songs on Every Event Planner's Playlist - 
Eventbrite](https://www.eventbrite.com/blog/event-planning-playlist-ds00/)
Music sets the mood and provides the soundtrack (literally) for a memorable and exciting time. While the right 
songs can enhance the experience, the wrong event music can throw off the vibe. For example, fast-paced songs 
probably aren't the best fit for a formal gala. And smooth jazz is likely to lull your guests at a motivational 
conference.

|The 75 Best Party Songs That Will Get Everyone Dancing - 
Gear4music](https://www.gear4music.com/blog/best-party-songs/)
The best party songs 1. "September" - Earth, Wind & Fire (1978) Quite possibly the best party song. An infectious 
mix of funk and soul, "September" is celebrated for its upbeat melody and "ba-dee-ya" chorus, making it a timeless 
dance favorite.

|agents-course/units/en/unit2/smolagents/tool_calling_agents ... - 
GitHub](https://github.com/huggingface/agents-course/blob/main/units/en/unit2/smolagents/tool_calling_agents.mdx)
from smolagents import ToolCallingAgent, DuckDuckGoSearchTool, HfApiModel agent = ToolCallingAgent (tools = 
|DuckDuckGoSearchTool ()], model = HfApiModel ()) agent. run ("Search for the best music recommendations for a 
party at the Wayne's mansion.")

|Wedding Party Entrance Songs to Get the Party Started - The Mansion 
...](https://mansiononmainstreet.com/wedding-party-entrance-songs-to-get-the-party-started/)
Best Wedding Party Entrance Songs. No matter what vibe you're going for, there are some wedding party entrance 
songs that are guaranteed to be a hit with people. From the latest music from Justin Timberlake to oldies but 
goodies, most of your guests will be familiar with the popular wedding party entrance songs listed below.

|200 Classic House Party Songs Everyone Knows | The Best ... - 
iSpyTunes](https://www.ispytunes.com/post/house-party-songs)
" Branded merchandise adds flair to any occasion, just like the perfect playlist. 200 classic house party songs 
everyone knows set the mood, bringing energy to every celebration. The best popular party hits keep guests dancing,
creating unforgettable moments. From throwback anthems to modern beats, a great selection ensures nonstop fun.

|Creating a playlist for cocktail hour and dinner service for ... - 
Reddit](https://www.reddit.com/r/wilco/comments/12kxks1/creating_a_playlist_for_cocktail_hour_and_dinner/)
Shakey Graves - Mansion Door ... Feel-good and Upbeat songs for a Kids' Campout Party ... A subreddit to ask for 
our provide recommendations of any relevant media based on other media people like - whether it be music, 
television, video games, movies, or anything else. This is a RECOMMENDATION ONLY sub!

|Passaic County Parks & Recreation: Music at the 
Mansion](https://passaiccountynj.myrec.com/info/activities/program_details.aspx?ProgramID=29909)
Thursdays from 7 to 9 PM the finest local bands will be playing music while In the Drink restaurant sells food and 
drinks on site. September 3rd: Norton Smull Band; Parking is limited at the Dey Mansion 209 Totowa Rd. Wayne, NJ. 
Overflow parking will be at the Preakness Valley Golf Course. You may drop off your guests at the Mansion first.

|Outsider to Wayne's music here, wheres a good place to 
start?](https://www.reddit.com/r/lilwayne/comments/p2csri/outsider_to_waynes_music_here_wheres_a_good_place/)
My rule to introduce Wayne is 3. Da Drought 3 Cárter 3 And No Ceilings 3. 3 projects with the number 3. Da Drought 
3, is praised as WAYNE's best mixtape. Tha Carter III, had a huge impact and may be Wayne's most successful album. 
No Ceilings 3, is newer so you can see how he's still doing his thing. Then whatever you want.

|Best 18 Songs for a Cozy and Sophisticated Dinner 
Party](https://www.wokewaves.com/posts/perfect-songs-for-intimate-dinner-party)
From smooth jazz to soulful ballads, these 18 songs are the perfect backdrop for meaningful conversations, 
delicious food, and 

[Step 0: Duration 1.16 seconds| Input tokens: 1,174 | Output tokens: 26]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "For a party at Wayne's mansion, consider a mix ofWest  │
│ Coast hip-hop and classic party hits. Based on the recommendations found, include tracks like 'Da Drought 3'    │
│ and 'Tha Carter 3' by Lil Wayne. For variety and to set different moods, you can also include classic party     │
│ songs such as 'September' by Earth, Wind & Fire and 200 classic house party songs that everyone knows from      │
│ iSpyTunes. This will ensure a wide appeal and provide a fun dance floor experience for everyone."}              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: For a party at Wayne's mansion, consider a mix ofWest Coast hip-hop and classic party hits. Based on 
the recommendations found, include tracks like 'Da Drought 3' and 'Tha Carter 3' by Lil Wayne. For variety and to 
set different moods, you can also include classic party songs such as 'September' by Earth, Wind & Fire and 200 
classic house party songs that everyone knows from iSpyTunes. This will ensure a wide appeal and provide a fun 
dance floor experience for everyone.

[Step 1: Duration 17.84 seconds| Input tokens: 3,476 | Output tokens: 154]

"For a party at Wayne's mansion, consider a mix ofWest Coast hip-hop and classic party hits. Based on the recommendations found, include tracks like 'Da Drought 3' and 'Tha Carter 3' by Lil Wayne. For variety and to set different moods, you can also include classic party songs such as 'September' by Earth, Wind & Fire and 200 classic house party songs that everyone knows from iSpyTunes. This will ensure a wide appeal and provide a fun dance floor experience for everyone."